# CityShadows: Mapping Trees
This notebook processes a JSON file with the following fields:

- fileName – Latitude, longitude, and degrees (lat,long_deg).
- objectType – Either "structure" or "vegetation".
- distance – Object distance from the camera (float).
- height – Estimated height of the object (float).
- width – Estimated width of the object (float).
- horizontalOffset – Horizontal position of the object in the image (-100 = leftmost, 0 = center, 100 = rightmost).
- isCut – Array indicating whether the object is partially cut off in the image.

The script generates tree polygons with height and width values taken from the JSON file, along with an estimated length. Based on the recorded distance, this polygon is projected onto its corresponding location on the map.

For this study, we used a subset of **Makati City, Metro Manila**. Building footprints from GeoFabrik (https://www.geofabrik.de/) were incorporated for visualization purposes only in this notebook.

## Libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import math
from shapely.affinity import rotate, translate
from shapely.geometry import box, Point

## Variables
### shp files

In [ ]:
small_makati = gpd.read_file("../../data/smaller_makati.shp")
buildings = gpd.read_file("../../data/gis_osm_buildings_a_free_1.shp")
less_makati_buildings = buildings[small_makati['geometry'].item().contains(buildings['geometry'])]

### json

In [ ]:
with open("output.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

trees_df = df[df['objectType'] == 'vegetation'].copy()

# Remove entries with height < 2m or > 20m and width > 20m
trees_df = trees_df[(trees_df['height'] >= 2) & (trees_df['height'] <= 20) & (trees_df['width'] <= 20)]

print(f"Found {len(trees_df)} trees after filtering")
print(f"Unique files: {trees_df['fileName'].nunique()}")
trees_df.head()

## Function Declaration
### Calculate tree position based on distance, offset, and camera configurations

In [ ]:
def calculate_tree_position(
    lat: float,
    lon: float,
    bearing: float,
    distance: float = None,              
    horizontal_offset: float = 0.0,       
    pitch_degrees: float = 20.0,        
    fov_degrees: float = 120.0,          
    camera_height: float = 2.45,
) -> Point:
    camera_xy = (
        gpd.GeoSeries([Point(lon, lat)], crs="EPSG:4326")
           .to_crs(epsg=3857)
           .iloc[0]
    )

    ratio = horizontal_offset / 100.0
    half_fov_rad = math.radians(fov_degrees / 2.0)
    angle_offset = math.degrees(math.atan(ratio * math.tan(half_fov_rad)))
    eff_bearing = bearing + angle_offset

    if distance <= camera_height:
        raise ValueError("Tree distance must be greater than camera height.")
    ground_distance = math.sqrt(distance**2 - camera_height**2)

    rad = math.radians(eff_bearing)
    dx = ground_distance * math.sin(rad)
    dy = ground_distance * math.cos(rad)

    return Point(camera_xy.x + dx, camera_xy.y + dy)

### Estimate tree length given width

In [ ]:
def estimate_length(row):
    w = row["width"]
    if w <= 10:
        return min(w * np.random.uniform(0.9, 1.1), 5)
    else:
        return min(w * 0.5, 5)

## Prep trees GeoDataFrame for GeoJSON export

In [ ]:
tree_dataset = gpd.GeoDataFrame(
    columns=['fileName', 'height', 'width', 'length', 'centerPoint', 'geometry'],
    geometry='geometry',
    crs='EPSG:4326'  
)
tree_dataset

## Process trees and project points based on their location

In [ ]:
tree_dataset = gpd.GeoDataFrame(
    tree_dataset,
    geometry='geometry',
    crs='EPSG:4326'
)

# Populate with center_point and calculated length
total = len(trees_df)
print(f"Starting to process {total} trees...\n")

for i, (_, row) in enumerate(trees_df.iterrows(), start=1):
    print(f"[{i}/{total}] Processing tree '{row['fileName']}'")
    
    # Parse camera info
    lat_lon, ang = row['fileName'].split('_')
    lat, lon = map(float, lat_lon.split(','))
    bearing = float(ang)
    
    h = float(row['height'])
    w = float(row['width'])
    off = float(row['horizontalOffset'])
    
    center_merc = calculate_tree_position(lat, lon, bearing, row['distance'], off)
    center_wgs = (
        gpd.GeoSeries([center_merc], crs="EPSG:3857")
           .to_crs(epsg=4326)
           .iloc[0]
    )
    
    length = estimate_length(row)
    
    tree_dataset.loc[len(tree_dataset)] = {
        'fileName':     row['fileName'],
        'height':       h,
        'width':        w,
        'length':       length,
        'centerPoint':  center_wgs,
        'geometry':     None
    }
    
    print(f"    -> Stored center at (lat={center_wgs.y:.6f}, lon={center_wgs.x:.6f}), length={length:.2f}m\n")

print(f"Done! {len(tree_dataset)} records in tree_dataset.")

### Visualize trees as points

In [ ]:
basemap_gdf = less_makati_buildings.to_crs(epsg=4326)
fig, ax = plt.subplots(figsize=(12, 12))

basemap_gdf.plot(
    ax=ax,
    color='lightgray',
    edgecolor='black',
    alpha=0.6
)

tree_dataset.set_geometry('centerPoint').plot(
    ax=ax,
    color='green',
    markersize=20,
    label='Trees'
)

ax.set_title('Tree Locations over Makati', fontsize=16)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()

plt.tight_layout()
plt.show()

## Create tree canopy rectangles

In [ ]:
gdf = tree_dataset.copy().drop(columns=['geometry'], errors='ignore')

gdf = gpd.GeoDataFrame(
    gdf,
    geometry='centerPoint',
    crs='EPSG:4326'
)

gdf_utm = gdf.to_crs(epsg=32651)

# Build rotated canopy rectangles (width × length)
rects = []
total = len(gdf_utm)
for idx, row in gdf_utm.iterrows():
    bearing = float(row['fileName'].split('_')[1])   # 0°=north, cw+
    cx, cy = row.centerPoint.x, row.centerPoint.y
    hw, hl = row.width/2.0, row.length/2.0

    # Axis-aligned rect around origin
    base = box(-hw, -hl, hw, hl)
    # Rotate CW by bearing (rotate() is CCW -> use –bearing)
    rot = rotate(base, -bearing, origin=(0, 0), use_radians=False)
    # Translate into place
    rects.append(translate(rot, xoff=cx, yoff=cy))

    print(f"[{idx+1}/{total}] Built canopy at UTM ({cx:.1f}, {cy:.1f}), bearing={bearing}")

# Assign rotated polygons and reproject back to WGS84
gdf_utm['geometry'] = rects
tree_dataset_final = gdf_utm.to_crs(epsg=4326).copy()

In [ ]:
tree_dataset_final = tree_dataset_final.set_geometry("geometry")
tree_dataset_final.set_geometry("geometry", inplace=True)
tree_dataset_final = tree_dataset_final.set_crs(epsg=32651, inplace=False)
tree_dataset_final = tree_dataset_final.to_crs(epsg=4326)
tree_dataset_final

## Visualization

In [ ]:
buildings = less_makati_buildings.to_crs(epsg=4326)
trees = tree_dataset_final  

fig, ax = plt.subplots(figsize=(12, 12))

buildings.plot(
    ax=ax,
    color='lightgray',
    edgecolor='black',
    alpha=0.5,
    linewidth=0.5,
    label='Buildings'
)

trees.plot(
    ax=ax,
    color='green',
    edgecolor='darkgreen',
    alpha=0.6,
    label='Tree Canopies'
)

ax.set_title('Tree Canopy Rectangles over Makati Buildings', fontsize=16)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()

plt.tight_layout()

output_path = 'tree_canopy_map.png'
plt.savefig(output_path, dpi=300)
print(f"Map saved as PNG to: {output_path}")

plt.show()

## Export GDF to GeoJSON file

In [ ]:
real = tree_dataset_final.copy()
real.drop(columns=['centerPoint'], inplace=True)
real.to_file("trees_dataset.geojson", driver="GeoJSON")